### **1.   Lire le fichier "X_train_uptade.csv" et "Y_trainCVw08PX.csv"**

In [50]:
# Importation des librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Chargement des fichiers '="X_train_uptade.csv" et "Y_trainCVw08PX.csv"
X_train = pd.read_csv('../../../../X_train.csv', index_col=0)
y_train = pd.read_csv('../../../../Y_train.csv', index_col=0)

# Afficher le nombre de lignes et de colonnes de chacun des deux fichiers
print("Le nombre de lignes et de colonnes de X_train est de :",X_train.shape,
      "\nLe nombre de lignes et de colonnes de y_train est de :", y_train.shape)

Le nombre de lignes et de colonnes de X_train est de : (84916, 4) 
Le nombre de lignes et de colonnes de y_train est de : (84916, 1)


### **2.   Fusion des fichiers "X_train_uptade.csv" et "Y_trainCVw08PX.csv"**




In [51]:
# Fusion avec merge des deux datasets
df = pd.merge(X_train, y_train, left_index=True, right_index=True)

### **3.   Afficher un aperçu des 10 premières lignes**






In [52]:
# Affichage des 5 premières lignes
display(df.head())

# Affichage des 5 dernières lignes
display(df.tail())


,designation,description,productid,imageid,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705


,designation,description,productid,imageid,prdtypecode
84911,The Sims [ Import Anglais ],NaN,206719094,941495734,40
84912,Kit piscine acier NEVADA déco pierre Ø 3.50m x...,<b>Description complète :</b><br />Kit piscine...,3065095706,1188462883,2583
84913,Journal Officiel De La Republique Francaise N°...,NaN,440707564,1009325617,2280
84914,Table Basse Bois De Récupération Massif Base B...,<p>Cette table basse a un design unique et con...,3942400296,1267353403,1560
84915,Gomme De Collection 2 Gommes Pinguin Glace Ver...,NaN,57203227,684671297,2522


### **4.   Fusion des colonnes Description et Designation**

In [53]:
# Créer une nouvelle colonne "description_complete" en concaténant les valeurs de "description" et "designation"
df['description_complete'] = df['designation'].fillna('') + ' ' + df['description'].fillna('')
df.drop(['description', 'designation'], axis=1, inplace=True)
df.head()

,productid,imageid,prdtypecode,description_complete
0,3804725264,1263597046,10,Olivia: Personalisiertes Notizbuch / 150 Seite...
1,436067568,1008141237,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2,201115110,938777978,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3,50418756,457047496,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...
4,278535884,1077757786,2705,La Guerre Des Tuques Luc a des id&eacute;es de...


### **5.   cleaning de description_complete**



In [55]:
#for text in df['description_complete'].head():
#    print(text)

In [56]:
from bs4 import BeautifulSoup


def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df['description_complete'] = df['description_complete'].apply(lambda x: remove_html_tags(x) if pd.notnull(x) else x)

df.head()


/var/folders/8_/rts9h4kn0g9ftjd7qn06h2t80000gn/T/ipykernel_18512/403255146.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


,productid,imageid,prdtypecode,description_complete
0,3804725264,1263597046,10,Olivia: Personalisiertes Notizbuch / 150 Seite...
1,436067568,1008141237,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2,201115110,938777978,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3,50418756,457047496,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...
4,278535884,1077757786,2705,La Guerre Des Tuques Luc a des idées de grande...


In [57]:
# Importation de la librairie worcloud et de la fonction WordCloud
from wordcloud import WordCloud
import nltk
'''nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')'''

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

# Fonction pour le prétraitement du texte
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les caractères spéciaux et les chiffres
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenisation
    words = word_tokenize(text)
    # Créer une liste de stopwords en français
    stopwords_french = stopwords.words('french')
    # Ajouter "X" et "x" à la liste de stopwords
    stopwords_french.extend(["Ap.", "Apr.", "GHz", "MHz", "USD", "a", "afin", "ah", "ai", "aie", "aient", "aies", "ait", "alors", "après", "as", "attendu",
                             "au", "au-delà", "au-devant", "aucun", "aucune", "audit", "auprès", "auquel", "aura", "aurai", "auraient", "aurais", "aurait",
                             "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autour", "autre", "autres", "autrui", "aux", "auxdites", "auxdits",
                             "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avons", "ayant", "ayez", "ayons", "b",
                             "bah", "banco", "ben", "bien", "bé", "c", "c'", "c'est", "c'était", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles",
                             "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "celà", "cent", "cents", "cependant", "certain", "certaine", "certaines", "certains",
                             "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "cf.", "cg", "cgr", "chacun", "chacune", "chaque", "chez", "ci", "cinq", "cinquante", "cinquante-cinq",
                             "cinquante-deux", "cinquante-et-un", "cinquante-huit", "cinquante-neuf", "cinquante-quatre", "cinquante-sept", "cinquante-six", "cinquante-trois", "cl", "cm",
                             "cm²", "comme", "contre", "d", "d'", "d'après", "d'un", "d'une", "dans", "de", "depuis", "derrière", "des", "desdites", "desdits", "desquelles", "desquels",
                             "deux", "devant", "devers", "dg", "différentes", "différents", "divers", "diverses", "dix", "dix-huit", "dix-neuf", "dix-sept", "dl", "dm", "donc", "dont",
                             "douze", "du", "dudit", "duquel", "durant", "dès", "déjà", "e", "eh", "elle", "elles", "en", "en-dehors", "encore", "enfin", "entre", "envers", "es", "est",
                             "et", "eu", "eue", "eues", "euh", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eûmes", "eût", "eûtes", "f", "fait", "fi",
                             "flac", "fors", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "g", "gr", "h", "ha", "han", "hein", "hem",
                             "heu", "hg", "hl", "hm", "hm³", "holà", "hop", "hormis", "hors", "huit", "hum", "hé", "i", "ici", "il", "ils", "j", "j'", "j'ai", "j'avais", "j'étais", "jamais",
                             "je", "jusqu'", "jusqu'au", "jusqu'aux", "jusqu'à", "jusque", "k", "kg", "km", "km²", "l", "l'", "l'autre", "l'on", "l'un", "l'une", "la", "laquelle", "le", "lequel",
                             "les", "lesquelles", "lesquels", "leur", "leurs", "lez", "lors", "lorsqu'", "lorsque", "lui", "lès", "m", "m'", "ma", "maint", "mainte", "maintes", "maints", "mais",
                             "malgré", "me", "mes", "mg", "mgr", "mil", "mille", "milliards", "millions", "ml", "mm", "mm²", "moi", "moins", "mon", "moyennant", "mt", "m²", "m³", "même", "mêmes",
                             "n", "n'avait", "n'y", "ne", "neuf", "ni", "non", "nonante", "nonobstant", "nos", "notre", "nous", "nul", "nulle", "nº", "néanmoins", "o", "octante", "oh", "on", "ont",
                             "onze", "or", "ou", "outre", "où", "p", "par", "par-delà", "parbleu", "parce", "parmi", "pas", "passé", "pendant", "personne", "peu", "plus", "plus_d'un", "plus_d'une",
                             "plusieurs", "pour", "pourquoi", "pourtant", "pourvu", "près", "puisqu'", "puisque", "q", "qu", "qu'", "qu'elle", "qu'elles", "qu'il", "qu'ils", "qu'on", "quand", "quant",
                             "quarante", "quarante-cinq", "quarante-deux", "quarante-et-un", "quarante-huit", "quarante-neuf", "quarante-quatre", "quarante-sept", "quarante-six", "quarante-trois", "quatorze",
                             "quatre", "quatre-vingt", "quatre-vingt-cinq", "quatre-vingt-deux", "quatre-vingt-dix", "quatre-vingt-dix-huit", "quatre-vingt-dix-neuf", "quatre-vingt-dix-sept", "quatre-vingt-douze",
                             "quatre-vingt-huit", "quatre-vingt-neuf", "quatre-vingt-onze", "quatre-vingt-quatorze", "quatre-vingt-quatre", "quatre-vingt-quinze", "quatre-vingt-seize", "quatre-vingt-sept",
                             "quatre-vingt-six", "quatre-vingt-treize", "quatre-vingt-trois", "quatre-vingt-un", "quatre-vingt-une", "quatre-vingts", "que", "quel", "quelle", "quelles", "quelqu'", "quelqu'un",
                             "quelqu'une", "quelque", "quelques", "quelques-unes", "quelques-uns", "quels", "qui", "quiconque", "quinze", "quoi", "quoiqu'", "quoique", "r", "revoici", "revoilà", "rien", "s", "s'",
                             "sa", "sans", "sauf", "se", "seize", "selon", "sept", "septante", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "si",
                             "sinon", "six", "soi", "soient", "sois", "soit", "soixante", "soixante-cinq", "soixante-deux", "soixante-dix", "soixante-dix-huit", "soixante-dix-neuf", "soixante-dix-sept", "soixante-douze",
                             "soixante-et-onze", "soixante-et-un", "soixante-et-une", "soixante-huit", "soixante-neuf", "soixante-quatorze", "soixante-quatre", "soixante-quinze", "soixante-seize", "soixante-sept",
                             "soixante-six", "soixante-treize", "soixante-trois", "sommes", "son", "sont", "sous", "soyez", "soyons", "suis", "suite", "sur", "sus", "t", "t'", "ta", "tacatac", "tandis", "te", "tel",
                             "telle", "telles", "tels", "tes", "toi", "ton", "toujours", "tous", "tout", "toute", "toutefois", "toutes", "treize", "trente", "trente-cinq", "trente-deux", "trente-et-un", "trente-huit",
                             "trente-neuf", "trente-quatre", "trente-sept", "trente-six", "trente-trois", "trois", "très", "tu", "u", "un", "une", "unes", "uns", "v", "vers", "via", "vingt", "vingt-cinq", "vingt-deux",
                             "vingt-huit", "vingt-neuf", "vingt-quatre", "vingt-sept", "vingt-six", "vingt-trois", "vis-à-vis", "voici", "voilà", "vos", "votre", "vous", "w", "x", "y", "z", "zéro", "à", "ç'", "ça", "ès",
                             "étaient", "étais", "était", "étant", "étiez", "étions", "été", "étée", "étées", "étés", "êtes", "être", "ô"])
    # Supprimer les stopwords
    words = [word for word in words if word not in stopwords_french]
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Prétraiter la colonne description_complete
df['clean_description_complete'] = df['description_complete'].apply(preprocess_text)

df.head()

,productid,imageid,prdtypecode,description_complete,clean_description_complete
0,3804725264,1263597046,10,Olivia: Personalisiertes Notizbuch / 150 Seite...,olivia personalisiertes notizbuch seiten punkt...
1,436067568,1008141237,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,journal art lart marche salon dart asiatique p...
2,201115110,938777978,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,grand stylet ergonomique bleu gamepad nintendo...
3,50418756,457047496,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...,peluche donald europe disneyland marionnette d...
4,278535884,1077757786,2705,La Guerre Des Tuques Luc a des idées de grande...,guerre tuques luc ides grandeur veut organiser...


#### **6 Traduction**
La traduction prend trop de temps pour être utilisable.

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# vectorisation des descriptions des produits
vectorizer = TfidfVectorizer(max_features = 5000)  # max_features choisit arbitrairement # Stopwords déjà fait
X = vectorizer.fit_transform(df['clean_description_complete']).toarray()
y = df['prdtypecode']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


'''from sklearn.utils import resample

# Identifier la classe majoritaire et la classe minoritaire
majority_class = df['prdtypecode'].value_counts().idxmax()
minority_classes = df['prdtypecode'].value_counts().idxmin()

# Séparer les données en classes majoritaires et minoritaires
majority_data = df[df['prdtypecode'] == majority_class]
minority_data = df[df['prdtypecode'] == minority_classes]'''

'''# Sous-échantillonner la classe majoritaire pour égaler la taille de la classe minoritaire
undersampled_majority_data = resample(majority_data, replace=False, n_samples=len(minority_data), random_state=42)

# Concaténer les classes minoritaires et sous-échantillonnées
undersampled_df = pd.concat([undersampled_majority_data, minority_data])

# Afficher les nouvelles proportions
print(undersampled_df['prdtypecode'].value_counts())
'''

# Undersampling aléatoire :

#from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids

#ru = RandomUnderSampler()
#X_ru, y_ru = ru.fit_resample(X_train, y_train)
# Afficher les nouvelles proportions
#print('Classes échantillon undersampled :', dict(pd.Series(y_ru).value_counts()))



"# Sous-échantillonner la classe majoritaire pour égaler la taille de la classe minoritaire\nundersampled_majority_data = resample(majority_data, replace=False, n_samples=len(minority_data), random_state=42)\n\n# Concaténer les classes minoritaires et sous-échantillonnées\nundersampled_df = pd.concat([undersampled_majority_data, minority_data])\n\n# Afficher les nouvelles proportions\nprint(undersampled_df['prdtypecode'].value_counts())\n"

Extraction d'un échantillon random (500 lignes)

In [59]:
from sklearn.model_selection import train_test_split

# Utiliser stratify pour maintenir la distribution des classes
X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Params pour la grid
param_grid = {'n_estimators': [10, 50, 100, 200],
              'max_features': ['sqrt', 'log2', None],
              'min_samples_split': range(2, 30, 2)}

# Init RF
rf = RandomForestClassifier()

# Init Grid
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs = -1)

# grid entrainement
grid_search.fit(X_train_sample, y_train_sample)

# Best_params
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

#### **7 Vectorization**
La plupart des algorithmes de machine learning se basent sur des données numériques. Nous voulons utiliser un random forest, pour ce faire nous avons besoin de convertir les valeurs textes en un format compréhensible par le model.
Pour ce faire nous pouvons utiliser TfidfVectorizer disponible dans scikit-learn. TF-IDF (Term Frequency-Inverse Document Frequency) est une technique populaire en machine learning lorsqu'il s'agit de vectorizer des données textuelles en valeurs numériques.

- Term Frequency (TF) est la mesure du taux d'apparition d'un mot dans un document.
- Inverse Document Frequency (IDF) est la mesure de l'importance d'un mot dans un jeu de données. Les mots présents fréquements dans l'ensemble des documents sont moins important que les mots présents dans peu de documents.
- TF-IDF est le produit de TF et de IDF

En utilisant TF-IDF nous représentons chaque document comme une vecteur de valeurs numérique où chaque dimencion représente un mot unique du jeu de données. C'est sur cette vectorization que nous allons pouvoir entrainer nos modèles.


In [15]:
'''from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


vectorizer = TfidfVectorizer(max_features=5000)  # max_features choisit arbitrairement
X = vectorizer.fit_transform(df_forest['clean_description_complete']).toarray()
y = df_forest['prdtypecode']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print("Classification Report:")
print(classification_report(y_test, y_pred))'''

Accuracy: 0.87
Classification Report:
              precision    recall  f1-score   support

        1180       0.79      0.98      0.87        45
        2583       0.98      0.78      0.87        55

    accuracy                           0.87       100
   macro avg       0.88      0.88      0.87       100
weighted avg       0.89      0.87      0.87       100



In [16]:
'''from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

# Create a pipeline with TfidfVectorizer and RandomForestClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier(random_state=42))])

# Define the parameter grid to search over
param_grid = {
'tfidf__max_features': [1000, 2000, 3000, 5000, 8000],  # [1000, 2000, 3000, 5000, 8000]
'rf__n_estimators': [50, 100, 200],  # Nombre d'arbres dans la forêt
'rf__max_features': ['sqrt', 'log2', None],  # Nombre de caractéristiques à considérer pour la meilleure division
'rf__min_samples_split': range(2, 30, 2)  # Nom du paramètre corrigé
}

#sqrt Matthieu
#log2 Caro
#None Tristan

# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs = -1) 

#F1-score
classification_report(y_test, y_pred)

# Fit the grid search to the data
grid_search.fit(df_forest['clean_description_complete'], df_forest['prdtypecode'])

# Print the best parameters and corresponding accuracy
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)
'''

Best Parameters: {'rf__max_features': 'sqrt', 'rf__min_samples_split': 2, 'rf__n_estimators': 200, 'tfidf__max_features': 1000}
Best Accuracy: 0.9400000000000001
